In [37]:
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing
import csv
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#filename = "cdf_score999.csv"
filename = "Crispr_Data.csv"
Scores = pd.read_csv(filename)

CCTop = Scores["CCTopSores"]
Cropit = Scores["CropitScores"]

MIT = Scores["MITscores"]

MITWeb = Scores["MITwebsite"]

CFD = Scores["cfdScore"]

Label = Scores["label"]
phastCons = Scores["scores"]
phastCons = phastCons.where(phastCons.notnull(), 0)
phylop = Scores["phylop"]
phylop = phylop.where(phylop.notnull(), 0)
Promoter = Scores["Promoter"]
Enhancer = Scores["Enhancer"]
segwayPromoter = Scores["segwayPromoter"]
segwayEnhancer = Scores["segwayEnhancer"]

#normsocres = np.transpose(np.vstack(( MIT, CFD, Enhancer)))
#normsocres = np.transpose(np.vstack((MIT,  CFD,  Promoter)))
#normsocres = np.transpose(np.vstack((MITWeb, CFD,  Enhancer)))
#normsocres = np.transpose(np.vstack((MIT, CFD,    phastCons)))
#normsocres = np.transpose(np.vstack((MITWeb, MIT, CCTop, Cropit, CFD)))
#normsocres = np.transpose(np.vstack((MITWeb, MIT, CFD)))
#normsocres = np.transpose(np.vstack(( Cropit, CFD,  Enhancer)))
#normsocres = np.transpose(np.vstack(( MIT,  CFD,   phylop)))
#normsocres = np.transpose(np.vstack(( MIT, Cropit, CFD )))
#normsocres = np.transpose(np.vstack((MIT,Cropit)))
normsocres = np.transpose(np.vstack((MITWeb, MIT, CCTop, Cropit, CFD, 
                                     phylop, phastCons)))


X_train, X_test, y_train, y_test = train_test_split(normsocres, Label, test_size=0.2, random_state=42)

bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm="SAMME", n_estimators=200)

cv = StratifiedKFold(n_splits=5)


In [39]:
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

i = 0
#plt.figure(figsize=(7, 7))  
sub_normsocres = normsocres[:,(0)].reshape(-1, 1)
#CFD,MITWeb, MIT, Cropit

for train, test in cv.split(sub_normsocres, Label):
    
    #AdaBoost
    scores = bdt.fit(sub_normsocres[train], Label[train]).decision_function(sub_normsocres[test])
    
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(Label[test], scores)
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC fold %d (AUC = %0.4f)' % (i, roc_auc))
    i += 1

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
print mean_auc

0.7419542256833065


In [1]:
0.862,0.862,0.742,0.862,0.742,0.742,0.871,0.862,0.862,0.862

NameError: name 'roc_auc_score' is not defined